# <div style="padding: 30px; color:white; margin:10; font-size:95%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b> Intel Review - Data Pre-Processing & NLP</b></div>

<div style="background-color: #3b3745; border-radius: 12px; padding: 20px; box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);">
    <h2 style="color: #F1A424; text-align: center;">Table of Contents</h2>
    <ul style="list-style: none; padding: 0;">
        <li><a href="#section-1" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">1.</span>  Importing Libraries</a></li>
        <li><a href="#section-2" style="color: white; text-decoration: none; display: flex; align-items: center; padding: 8px 15px; border-radius: 6px; transition: background-color 0.3s;"><span style="margin-right: 10px; font-weight: bold; color: #F1A424;">2.</span> Data Pre-Processing</a></li>
    </ul>
</div>

<a id="section-1"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>1 |</span></b> <b>  Importing Libraries</b></div>

In [1]:
import pandas as pd
from langdetect import detect, LangDetectException
from googletrans import Translator
import numpy as np
import re
import string

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [2]:
%pip install pandas langdetect googletrans==4.0.0-rc1

  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached googletrans-4.0.0rc1-py3-none-any.whl
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached h2-3.2.0-py2.py3-none-any.whl.metadata (32 kB)
  Using cached hyperframe-5.2.0-py2.py3-none-any.whl.metadata (7.2 kB)
  Using cached hpack-3.0.0-py2.py3-none-any.whl.metadata (7.0 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
Using cached h2-3.2.0-py2.py3-none-any.whl (65 kB)
Using cached rfc3986-1.5.0-py2.py3-none-any.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.2.0 requires httpx>=0.25.0, but you have httpx 0.13.3 which is incompatible.


<a id="section-1=2"></a>
# <div style="padding: 30px; color:white; margin:10; font-size:75%; text-align:left; display:fill; border-radius:10px; background-color:#3b3745"><b><span style='color:#F1A424'>2 |</span></b> <b>  Data Pre-Processing</b></div>

<div style="padding: 20px; color:white; margin:5; font-size:100%; text-align:left; display:fill; border-radius:5px; background-color:#3b3745">
    <h2 style="font-size: 150%; margin-bottom: 10px;">Steps to Pre-Processing :</h2>
    <b>1. Translating if Review is not English</b> <br>
    <b>2. Removing Numbers if exist</b> <br>
    <b>3. Removing Special Characters & Punctuations</b> <br>
    <b>4. Removing Extra Spaces</b> <br>
    <b>5. Converting to Lowercase</b> <br>
    <b>6. Removing Duplicates</b>
</div>


In [11]:
translator = Translator()

def detect_and_translate(text):
    text = text.strip()  
    if not text:
        return text  
    
    try:
        # Clean text further (remove non-printable characters)
        text = ''.join(filter(lambda x: x in string.printable, text))
        
        # Detect language
        lang = detect(text)
        print(f"Detected language: {lang} for text: {text[:30]}...")  
        
        # Translate if not English
        if lang != 'en':
            translated = translator.translate(text, dest='en')
            print(f"Translated text: {translated.text[:30]}...")  
            return translated.text
        else:
            return text
    
    except LangDetectException as e:
        print(f"LangDetectException: {e}. Text: {text}")  
        return text
    
    except Exception as e:
        print(f"Error: {e}. Text: {text}")  
        return text

def clean_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Convert to lowercase
    text = text.lower()
    return text

def clean_data(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    if 'Review' not in df.columns:
        raise ValueError("The input file does not contain a 'Review' column.")
    
    # Detect and translate non-English text in the 'Review' column
    df['Review'] = df['Review'].apply(lambda x: detect_and_translate(str(x)) if isinstance(x, str) else x)
    
    # Clean text in the 'Review' column
    df['Review'] = df['Review'].apply(lambda x: clean_text(x) if isinstance(x, str) else x)
    
    # Handle missing values
    for column in df.columns:
        if np.issubdtype(df[column].dtype, np.number):
            df[column] = df[column].fillna(df[column].mean())
        else:
            df[column] = df[column].fillna(df[column].mode()[0])
    
    # Remove duplicates
    df = df.drop_duplicates()
    
    return df

file_path = '../5-data/all-products-data.csv'  
cleaned_df = clean_data(file_path)

if cleaned_df.empty:
    print("Warning: The cleaned DataFrame is empty.")

# Save the cleaned dataframe to a new CSV file
cleaned_df.to_csv('../5-data/transuluted-all-products-data.csv', index=False)

print("Data cleaning and translation complete. Cleaned data saved to 'cleaned_file.csv'.")


LangDetectException: No features in text.. Text: ???????????
Data cleaning and translation complete. Cleaned data saved to 'cleaned_file.csv'.


                                   This concludes the analysis presented in this notebook.
